In [2]:
# Import libraries
import tensorflow as tf
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
# Import Warnings
import warnings

warnings.filterwarnings('ignore')
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
# from sklearn.cross_validation import train_test_split
# Import tensorflow as the backend for Keras

from keras import backend as K

K.set_image_data_format('channels_first')
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
# from keras.optimizers import SGD,RMSprop,adam
from keras.callbacks import TensorBoard
# Import required libraries for cnfusion matrix
from sklearn.metrics import classification_report, confusion_matrix
import itertools

PATH = os.getcwd()
# Define data path
data_path = '../input/images-dataset/data/data'
data_dir_list = os.listdir(data_path)
data_dir_list

img_rows = 128
img_cols = 128
num_channel = 1
num_epoch = 100
# Define the number of classes
num_classes = 7
img_data_list = []
for dataset in data_dir_list:
    img_list = os.listdir(data_path + '/' + dataset)
    print('Loaded the images of dataset-' + '{}\n'.format(dataset))
    for img in img_list:
        input_img = cv2.imread(data_path + '/' + dataset + '/' + img)
        input_img = cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)
        input_img_resize = cv2.resize(input_img, (128, 128))
        img_data_list.append(input_img_resize)

img_data = np.array(img_data_list)
img_data = img_data.astype('float32')
img_data /= 255
print(img_data.shape)

if num_channel == 1:
    if K.image_data_format() == 'channels_first':
        img_data = np.expand_dims(img_data, axis=1)
        print(img_data.shape)
    else:
        img_data = np.expand_dims(img_data, axis=4)
        print(img_data.shape)

else:
    if K.image_data_format() == 'channels_first':
        img_data = np.rollaxis(img_data, 3, 1)
        print(img_data.shape)

num_classes = 7
num_of_samples = img_data.shape[0]
labels = np.ones((num_of_samples,), dtype='int64')
labels[0:365] = 0
labels[365:567] = 1
labels[567:987] = 2
labels[987:1189] = 3
labels[1189:1399] = 4
labels[1399:1601] = 5
labels[1601:1803] = 6
names = ['bike', 'cars', 'cats', 'dogs', 'flowers', 'horses', 'human']

Y = np_utils.to_categorical(labels, num_classes)

x, y = shuffle(img_data, Y, random_state=2)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)

print("X_train shape = {}".format(X_train.shape))
print("X_test shape = {}".format(X_test.shape))

image = X_train[1203, :].reshape((128, 128))
plt.imshow(image)
plt.show()

# Initialising the input shape
input_shape = img_data[0].shape
# Design the CNN Sequential model
cnn_model = Sequential([
    Convolution2D(32, 3, 3, padding='same', activation='relu', input_shape=input_shape),
    Convolution2D(32, 3, 3, activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.5),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

cnn_model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=["accuracy"])

cnn_model.summary()

hist = cnn_model.fit(X_train, y_train, batch_size=16, epochs=num_epoch, verbose=1, validation_data=(X_test, y_test))
train_loss = hist.history['loss']
val_loss = hist.history['val_loss']
train_acc = hist.history['accuracy']
val_accuracy = hist.history['val_accuracy']
xc = range(num_epoch)

plt.figure(1, figsize=(10, 5))
plt.plot(xc, train_loss)
plt.plot(xc, val_loss)
plt.xlabel('Number of Epochs')
plt.ylabel('Loss')
plt.title('Train Loss vs Validation Loss')
plt.grid(True)
plt.legend(['Train Loss', 'Validation Loss'])
plt.style.use(['classic'])

plt.figure(2, figsize=(10, 5))
plt.plot(xc, train_acc)
plt.plot(xc, val_accuracy)
plt.xlabel('Number of Epochs')
plt.ylabel('Accuracy')
plt.title('Train Accuracy vs Validation Accuracy')
plt.grid(True)
plt.legend(['Train Accuracy', 'Validation Accuracy'], loc=4)
plt.style.use(['classic'])

score = cnn_model.evaluate(X_test, y_test, verbose=0)
print('Test Loss:', score[0])
print('Test Accuracy:', score[1])

test_image = X_test[0:1]
print(test_image.shape)
print(cnn_model.predict(test_image))
# print(cnn_model.predict_classes(test_image))
predict_x = cnn_model.predict(test_image)
classes_x = np.argmax(predict_x, axis=1)
print(y_test[0:1])

image = test_image.reshape((128, 128))
plt.imshow(image)
plt.show()

test_img = cv2.imread('../input/images-dataset/data/human/rider-104.jpg')
test_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)
test_img = cv2.resize(test_img, (128, 128))
test_img = np.array(test_img)
test_img = test_img.astype('float32')
test_img /= 255
print(test_img.shape)

if num_channel == 1:
    if K.image_data_format() == 'channels_first':
        test_img = np.expand_dims(test_img, axis=0)
        test_img = np.expand_dims(test_img, axis=0)
        print(test_img.shape)
    else:
        test_img = np.expand_dims(test_img, axis=3)
        test_img = np.expand_dims(test_img, axis=0)
        print(test_img.shape)

else:
    if K.image_data_format() == 'channels_first':
        test_img = np.rollaxis(test_img, 2, 0)
        test_img = np.expand_dims(test_img, axis=0)
        print(test_img.shape)
    else:
        test_img = np.expand_dims(test_img, axis=0)
        print(test_img.shape)

# Predicting the test image
print((cnn_model.predict(test_img)))
print(np.argmax(cnn_model.predict(test_img), axis=-1))


def get_featuremaps(cnn_model, layer_idx, X_batch):
    get_activations = K.function([cnn_model.layers[0].input, K.learning_phase()],
                                 [cnn_model.layers[layer_idx].output, ])
    activations = get_activations([X_batch, 0])
    return activations


tf.compat.v1.disable_eager_execution()
layer_num = 3
filter_num = 0
activations = get_featuremaps(cnn_model, int(layer_num), test_img)
print(np.shape(activations))

feature_maps = activations[0][0]
print(np.shape(feature_maps))

if K.image_data_format() == 'channels_first':
    feature_maps = np.rollaxis((np.rollaxis(feature_maps, 2, 0)), 2, 0)
print(feature_maps.shape)

fig = plt.figure(figsize=(16, 16))
plt.imshow(feature_maps[:, :, filter_num], cmap='gray')
plt.savefig("featuremaps-layer-{}".format(layer_num) + "-filternum-{}".format(filter_num) + '.jpg')
num_of_featuremaps = feature_maps.shape[2]
fig = plt.figure(figsize=(16, 16))
plt.title("featuremaps-layer-{}".format(layer_num))
subplot_num = int(np.ceil(np.sqrt(num_of_featuremaps)))
for i in range(int(num_of_featuremaps)):
    ax = fig.add_subplot(subplot_num, subplot_num, i + 1)
    # ax.imshow(output_image[0,:,:,i],interpolation='nearest' ) #to see the first filter
    ax.imshow(feature_maps[:, :, i], cmap='gray')
    plt.xticks([])
    plt.yticks([])
    plt.tight_layout()
plt.show()

# Print the confusion matrix
Y_pred = cnn_model.predict(X_test)
print(Y_pred)
y_pred = np.argmax(Y_pred, axis=1)
print(y_pred)
target_names = ['Class 0 (flowers)', 'Class 1 (cars)', 'Class 2 (cats)', 'Class 3 (horses)',
                'Class 4 (human)', 'Class 5 (bike)', 'Class 6 (dogs)']
print(classification_report(np.argmax(y_test, axis=1), y_pred, target_names=target_names))

print('Confusion Matrix \n')
print(confusion_matrix(np.argmax(y_test, axis=1), y_pred))


def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float32') / cm.sum(axis=1)[:, np.newaxis]
        print("Confusion matrix with Normalization")
    else:
        print('Confusion matrix without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


# Compute confusion matrix
cnf_matrix = (confusion_matrix(np.argmax(y_test, axis=1), y_pred))
np.set_printoptions(precision=2)
plt.figure()

plot_confusion_matrix(cnf_matrix, classes=target_names,
                      title='Confusion Matrix without Normalisation')
plt.show()

plot_confusion_matrix(cnf_matrix, classes=target_names, normalize=True,
                      title='Normalized Confusion Matrix')
plt.figure()
plt.show()